In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [2]:
df1 = pd.read_csv("results/tumor_22wk.txt", sep="\t", header=None)
df2 = pd.read_csv("results/control_22wk.txt", sep="\t", header=None)

NameError: name 'pd' is not defined

In [129]:
col_names = ['chrom', 'pos', 'ref', 'alt', 'S1_GT', 'S1_AD', 'S1_AF', 'S2_GT', 'S2_AD', 'S2_AF', 'S3_GT', 'S3_AD', 'S3_AF', 'S4_GT', 
'S4_AD', 'S4_AF', 'S5_GT', 'S5_AD', 'S5_AF', 'S6_GT', 'S6_AD', 'S6_AF', 'S7_GT', 'S7_AD', 'S7_AF', 'S8_GT', 'S8_AD', 'S8_AF', 'S9_GT', 
'S9_AD', 'S9_AF', 'S10_GT', 'S10_AD', 'S10_AF', 'S11_GT', 'S11_AD', 'S11_AF', 'S12_GT', 'S12_AD', 'S12_AF', 'S12_GT', 'S12_AD', 'S12_AF']
# change later to account for any len with  print(f"test{i}")
def add_names(df):
    col_list =[]
    for i in range(df.shape[1]):
        col_list.append(col_names[i])
    df.columns = col_list
    return df

add_names(df1)
add_names(df2)

,chrom,pos,ref,alt,S1_GT,S1_AD,S1_AF,S2_GT,S2_AD,S2_AF,S3_GT,S3_AD,S3_AF,S4_GT,S4_AD,S4_AF,S5_GT,S5_AD,S5_AF
0,chr10,3000812,G,A,./.,.,1,1/1,"0,2",1,./.,.,1,./.,.,1,./.,.,1
1,chr10,3000887,G,A,./.,.,1,1/1,"0,2",1,./.,.,1,./.,.,1,1/1,"0,2",1
2,chr10,3103996,T,A,./.,.,1,./.,.,1,./.,.,1,1/1,"0,4",1,./.,.,1
3,chr10,3104784,A,G,./.,.,1,1/1,"0,2",1,./.,.,1,./.,.,1,./.,.,1
4,chr10,3104802,C,T,./.,.,1,1/1,"0,2",1,./.,.,1,./.,.,1,./.,.,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636367,chrY,2901524,TGTGCCTG,GGTGGCTC,./.,.,1,./.,.,1,./.,.,1,1/1,"0,2",1,./.,.,1
636368,chrY,2901581,G,T,./.,.,1,./.,.,1,./.,.,1,1/1,"0,4",1,./.,.,1
636369,chrY,2901642,A,G,./.,.,1,./.,.,1,./.,.,1,1/1,"0,2",1,./.,.,1
636370,chrY,2901648,C,A,./.,.,1,./.,.,1,./.,.,1,1/1,"0,2",1,./.,.,1


In [130]:
#fixing 1s and .s 

def fix_dots(df):
    for i in range(1, int((df.shape[1]-4)/3)+1, 1):
        gt_col = f"S{i}_GT"
        af_col = f"S{i}_AF"
        df.loc[df[gt_col] == './.', af_col] = '0'
    return df
fix_dots(df1)
fix_dots(df2)

,chrom,pos,ref,alt,S1_GT,S1_AD,S1_AF,S2_GT,S2_AD,S2_AF,S3_GT,S3_AD,S3_AF,S4_GT,S4_AD,S4_AF,S5_GT,S5_AD,S5_AF
0,chr10,3000812,G,A,./.,.,0,1/1,"0,2",1,./.,.,0,./.,.,0,./.,.,0
1,chr10,3000887,G,A,./.,.,0,1/1,"0,2",1,./.,.,0,./.,.,0,1/1,"0,2",1
2,chr10,3103996,T,A,./.,.,0,./.,.,0,./.,.,0,1/1,"0,4",1,./.,.,0
3,chr10,3104784,A,G,./.,.,0,1/1,"0,2",1,./.,.,0,./.,.,0,./.,.,0
4,chr10,3104802,C,T,./.,.,0,1/1,"0,2",1,./.,.,0,./.,.,0,./.,.,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636367,chrY,2901524,TGTGCCTG,GGTGGCTC,./.,.,0,./.,.,0,./.,.,0,1/1,"0,2",1,./.,.,0
636368,chrY,2901581,G,T,./.,.,0,./.,.,0,./.,.,0,1/1,"0,4",1,./.,.,0
636369,chrY,2901642,A,G,./.,.,0,./.,.,0,./.,.,0,1/1,"0,2",1,./.,.,0
636370,chrY,2901648,C,A,./.,.,0,./.,.,0,./.,.,0,1/1,"0,2",1,./.,.,0


In [131]:
cols_to_keep = ['chrom', 'pos', 'ref', 'alt', 'S1_AD', 'S2_AD', 'S3_AD', 'S4_AD', 'S5_AD', 'S6_AD', 'S7_AD', 'S8_AD', 'S9_AD', 'S10_AD', 'S11_AD', 'S12_AD', 'S13_AD']
def drop_cols(df):
    for col in df.columns:
        if col not in cols_to_keep:
            df.drop(columns=[col], inplace=True)
    return df

drop_cols(df1)
drop_cols(df2)

,chrom,pos,ref,alt,S1_AD,S2_AD,S3_AD,S4_AD,S5_AD
0,chr10,3000812,G,A,.,"0,2",.,.,.
1,chr10,3000887,G,A,.,"0,2",.,.,"0,2"
2,chr10,3103996,T,A,.,.,.,"0,4",.
3,chr10,3104784,A,G,.,"0,2",.,.,.
4,chr10,3104802,C,T,.,"0,2",.,.,.
...,...,...,...,...,...,...,...,...,...
636367,chrY,2901524,TGTGCCTG,GGTGGCTC,.,.,.,"0,2",.
636368,chrY,2901581,G,T,.,.,.,"0,4",.
636369,chrY,2901642,A,G,.,.,.,"0,2",.
636370,chrY,2901648,C,A,.,.,.,"0,2",.


In [132]:
def get_num(df, col_name):
    num = []  
    for item in col_name:
        if not item or item in ('.', './.'):
            num.append(0)
        else:
            parts = item.split(',')
            numbers = [int(x) if x != '.' else 0 for x in parts]
            num.append(sum(numbers[1:]) if len(numbers) > 1 else 0)
    return num

def get_denom(df, col_name):
    denom = []    
    for item in col_name:
        if not item or item in ('.', './.'):
            denom.append(0)
        else:
            parts = item.split(',')
            numbers = [int(x) if x != '.' else 0 for x in parts]
            denom.append(sum(numbers))
    return denom

def sorting(df):
    nums = []
    denoms = []
    for i in range(1, df.shape[1]-3, 1):
        ad_list = df[f"S{i}_AD"].tolist()
        nums.append(get_num(df, ad_list))
        denoms.append(get_denom(df, ad_list))

    nums_col_sum = [sum(col) for col in zip(*nums)]
    denoms_col_sum = [sum(col) for col in zip(*denoms)]
    ratios = []
    for i in range(len(nums_col_sum)):
        ratios.append(nums_col_sum[i]/denoms_col_sum[i])
    df["snp_rate"] = ratios
    df["snp_cnt"] = denoms_col_sum
    final_cols_to_keep = ['chrom', 'pos', 'ref', 'alt', 'snp_rate', 'snp_cnt']
    return df[final_cols_to_keep]

df1 = sorting(df1)
df2 = sorting(df2)

In [133]:
df2.head()

,chrom,pos,ref,alt,snp_rate,snp_cnt
0,chr10,3000812,G,A,1.0,2
1,chr10,3000887,G,A,1.0,4
2,chr10,3103996,T,A,1.0,4
3,chr10,3104784,A,G,1.0,2
4,chr10,3104802,C,T,1.0,2


In [134]:
df1.head()

,chrom,pos,ref,alt,snp_rate,snp_cnt
0,chr10,3000306,ATTTCA,"TTTTTG,TTTTCG",1.0,10
1,chr10,3000330,T,C,1.0,6
2,chr10,3000335,G,A,1.0,7
3,chr10,3000489,C,G,1.0,23
4,chr10,3000504,G,A,1.0,20


In [135]:
dfm = pd.merge(df1, df2, on=['chrom','pos'], how='left')
dfm.head()

,chrom,pos,ref_x,alt_x,snp_rate_x,snp_cnt_x,ref_y,alt_y,snp_rate_y,snp_cnt_y
0,chr10,3000306,ATTTCA,"TTTTTG,TTTTCG",1.0,10,NaN,NaN,NaN,NaN
1,chr10,3000330,T,C,1.0,6,NaN,NaN,NaN,NaN
2,chr10,3000335,G,A,1.0,7,NaN,NaN,NaN,NaN
3,chr10,3000489,C,G,1.0,23,NaN,NaN,NaN,NaN
4,chr10,3000504,G,A,1.0,20,NaN,NaN,NaN,NaN


In [136]:
dfm = dfm.fillna(0)
dfm

,chrom,pos,ref_x,alt_x,snp_rate_x,snp_cnt_x,ref_y,alt_y,snp_rate_y,snp_cnt_y
0,chr10,3000306,ATTTCA,"TTTTTG,TTTTCG",1.0,10,0,0,0.0,0.0
1,chr10,3000330,T,C,1.0,6,0,0,0.0,0.0
2,chr10,3000335,G,A,1.0,7,0,0,0.0,0.0
3,chr10,3000489,C,G,1.0,23,0,0,0.0,0.0
4,chr10,3000504,G,A,1.0,20,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
761108,chrY,2899163,A,G,1.0,2,0,0,0.0,0.0
761109,chrY,2899173,A,T,1.0,4,0,0,0.0,0.0
761110,chrY,2899197,T,C,1.0,4,0,0,0.0,0.0
761111,chrY,2899217,A,T,1.0,4,0,0,0.0,0.0


In [137]:
dfm[(dfm['snp_cnt_x'] > 50) & (dfm['snp_rate_x'] > 0.8) & (dfm['snp_rate_y'] < 0.1)]

,chrom,pos,ref_x,alt_x,snp_rate_x,snp_cnt_x,ref_y,alt_y,snp_rate_y,snp_cnt_y
1445,chr10,7501102,C,GCAT,1.000000,78,0,0,0.0,0.0
1446,chr10,7501120,A,C,1.000000,134,0,0,0.0,0.0
1447,chr10,7501133,T,G,1.000000,145,0,0,0.0,0.0
1450,chr10,7501333,TTTTG,T,1.000000,55,0,0,0.0,0.0
1451,chr10,7501380,T,G,1.000000,88,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
744618,chrX,7315346,G,A,1.000000,51,0,0,0.0,0.0
749892,chrX,64031992,ATCGTCC,"GTCCTCA,ATCCTCA",0.860215,93,0,0,0.0,0.0
753603,chrX,97922028,G,C,1.000000,53,0,0,0.0,0.0
758793,chrX,148708080,C,T,1.000000,75,0,0,0.0,0.0


In [138]:
dfm[(dfm['snp_cnt_x'] > 50) & (dfm['snp_rate_x'] > 0.8) & (dfm['snp_rate_y'] < 0.1) & (dfm['snp_cnt_y'] > 30)]

,chrom,pos,ref_x,alt_x,snp_rate_x,snp_cnt_x,ref_y,alt_y,snp_rate_y,snp_cnt_y
255057,chr17,34136814,CATAT,"TCATC,CAAAT",1.000000,59,CATAT,TCATC,0.077869,244.0
452658,chr3,69768915,C,T,0.966667,120,C,T,0.090476,210.0
453092,chr3,75680521,G,A,0.826531,98,G,A,0.057143,35.0
454443,chr3,82853602,C,T,0.900990,101,C,T,0.058824,34.0
513192,chr4,137521063,AT,A,0.895604,364,A,C,0.060606,33.0
515558,chr4,139205193,G,A,0.995960,495,GGGG,CCCC,0.062016,129.0
530076,chr5,9100843,C,T,0.976000,125,C,T,0.052632,38.0
606470,chr6,135087832,ACCC,"CCCCCT,ACCCT",1.000000,69,A,C,0.070175,57.0
